In [1]:
from Gemini.GeminiPredict import GeminiPredict
from Generation.Equation_BaseDataset import Equation, BaseDataset
# from Postprocessing.
from Scraping.Scraper import Scraper

import os
import google.generativeai as genai

c:\Users\kyanj\anaconda3\envs\fyp_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Gemini

In [2]:
gp = GeminiPredict("GEMINI_API_KEY", "Data/gemini_test_4", "tunedModels/mmltopython4-f3fuppiemnq9")
gp.evaluate_bleu()

API key loaded successfully!


Generating Predictions: 100%|██████████| 50/50 [01:04<00:00,  1.29s/row]

SacreBLEU Score: 91.88131484150725


In [4]:
for model in genai.list_tuned_models():
    print(model.name)

tunedModels/mmltopython1-si6b5lpywfhi
tunedModels/mmltopython2-rq77o9de7x0p
tunedModels/mmltopython4-f3fuppiemnq9


### Generation

In [ ]:
eq = Equation()


In [4]:
class GeminiDataset(BaseDataset):
    def __init__(self, num, filepath):
        super().__init__(num, filepath)

    def get_columns(self):
        return ["text_input", "output"]
    
gd = GeminiDataset(50, "Data/gemini_test_4.csv")
gd.create()

Generating dataset: 100%|██████████| 50/50 [00:00<00:00, 66.17it/s]


### Postprocessing

### Scraping

In [ ]:
sc = Scraper()

### Pseudo Main

- choose DOI to scrape from
- use Scraper class to get MathML equations from paper and save to a txt file
- (also scrape tables for constants)

